## Greedy Best-First Search

To formaulate the scheudule as a tree search problem, we will first have to define the node and the vertices.

To simplify the case, we can assume that each timeslot represents a level of a node. As such, an increment of depth from depth = $t$ represent the $t+1$ timeslot.

To account for flights having to divert, we will need to construct the state as a tuple of `new_schedule` and `diverted_flight`. The `new_schedule` is a dictionary of keys representing flight code and the value representing the time slots. If the flight has to be diverted, the timeslot assigned would equate to `pd.to_datetime(0)`

In [1]:
# import the necessary packages
import pandas as pd
import numpy as np
import sys
import itertools

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



/Users/Tra_FIT/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# load the data
df = pd.read_csv("./data/21DEC2023_AMS_processed.csv", parse_dates=['time_sch', 'time_act'])
df.tail(5)

,time_sch,time_act,code,dest,stat,orig,pass_load,time_diff
1195,2023-12-21 23:25:00,2023-12-22 00:21:00,KL 1608 KLM,Amsterdam,BAGGAGE ON BELT,Rome (FCO),309,-83040.0
1196,2023-12-21 23:30:00,2023-12-22 00:29:00,HV 5806 Transavia,Amsterdam,BAGGAGE ON BELT,Thessaloniki (SKG),310,-82860.0
1197,2023-12-21 23:40:00,2023-12-22 00:27:00,HV 6902 Transavia,Amsterdam,BAGGAGE ON BELT,Dubai International (DXB),316,-83580.0
1198,2023-12-21 23:50:00,2023-12-22 00:52:00,HV 5666 Transavia,Amsterdam,BAGGAGE ON BELT,Las Palmas de Gran Canaria (LPA),347,-82680.0
1199,2023-12-21 23:55:00,2023-12-22 01:00:00,HV 5218 Transavia,Amsterdam,BAGGAGE ON BELT,Catania (CTA),304,-82500.0


## Formulate the tree
We will consider the general case where the class `problem` initialises with the following attributes that can be customized for any specific flight re-scheduling problem:
* n-runway
* time of operation resumption
* maximum delay of each flight
* duration of each time slot

In [40]:
class reschedule_problem:

    def __init__(self,df, n_runway = 1, resume_hour = 23, resume_min = 0,\
                timeslot_dur = 5, max_delay = 120):
        # customized attributes of the problem
        self.df = df
        self.move = df['code']
        self.n_runway = n_runway
        self.hour = resume_hour 
        self.min = resume_min 
        self.time_slot_duration = timeslot_dur
        self.max_delay = max_delay
        # parsing the attribute for more useful format
        self.date = self.df.loc[0,'time_sch'].day
        self.month = self.df.loc[0,'time_sch'].month
        self.year = self.df.loc[0,'time_sch'].year
        self.initial = pd.DataFrame(columns = ["time_new","util"], index= []) # to store the solution

    def actions(self, state):
        """return a list of n_runway flight(s) that has/have is expected to land
        and is not assigned a time slot under the state"""
        # get the list of unassigned flight where the time_sch is (over)due under current time slot 
        assigned_flight = list([flight for flight in state.index])
        unassigned_flight = self.df.query("code not in @assigned_flight").copy()
        # subset the flights that are not in time yet
        year, month, date, hour, min = self.parse_state_time(state)
        current_time = pd.to_datetime(f"{year} {month} {date} {hour}:{min}")
        min_time = pd.to_datetime(f"{year} {month} {date} {hour}:{min}") - pd.Timedelta(minutes = self.max_delay)
        unassigned_flight['divert'] = unassigned_flight.apply(lambda flight: flight['time_sch'] < min_time  ,axis = 1)
        unassigned_flight['to_sch'] = unassigned_flight.apply(lambda flight: flight['time_sch'] <=current_time  ,axis = 1)

        # return the pd.Series of unassigned flight that has passed its schedule time
        flight_to_assign = unassigned_flight[unassigned_flight['to_sch'] & ~(unassigned_flight['divert'])]['code']
        flight_to_assign = [comb for n in range(1, self.n_runway+1) for comb in itertools.combinations(flight_to_assign, n)]
        # flight to divert
        flight_diverted = unassigned_flight[unassigned_flight['divert']]['code']
        
        return flight_to_assign, flight_diverted # pd.Series
        
    def parse_state_time(self,state):
        """Use to get the value in the schedule and 
        parse the time of the next time slot"""
        if len(state) <=0: 
            year = self.year
            month = self.month
            date = self.date
            hour = self.hour
            min = self.min

        else:
            timestr = state['time_new'].iloc[-1] # get the time_sch
            year, month, date, time = timestr.split(" ")
            hour , min = time.split(":")
            # get the current time
            min = int(min)
            min += self.time_slot_duration
            if min // 60 <= 1:
                hour = int(hour) + min // 60
                min = min % 60
                if hour // 24 <= 1:
                    date = int(date) + hour // 24
                    hour = hour % 24
                
        return year, month, date, hour,min
    
    def result(self, state, flight_diverted:list, flights:list):
        """return the state in a form of dictionary that is the result of a given move"""
        # !!! think of a way to iterate through the time slot even if no flight assigned
        year, month, date, hour, min = self.parse_state_time(state)
        for fl in flights:
            time_sch = f"{year} {month} {date} {hour:02d}:{min:02d}"
            # get the utility
            util = self.compute_util(fl, year, month, date, hour, min)
            # print(f"Scheudling {flights} for {time_sch}")
            state.loc[fl] = [time_sch, util]
    
        # add the diverted flight into the state
        for fl in flight_diverted:
            util = self.compute_util(fl,1970, 0,0,0,0)
            state.loc[fl] = ["1970 01 01 00:00", util]
            # print(f"Diverting {flights}")

        state = state.sort_values("time_new")

        return state # pd.DataFrame
    
    def compute_util(self, flcode, year, month, date, hour, min):
        """Compute the utility of a given rescheduled flight"""
        if flcode is None:
            return 0 
        elif year != 1970:
            time_sch_org = self.df.query("code == @flcode")['time_sch'] # type pd Series
            time_sch_new = pd.to_datetime(f"{year} {month} {date} {hour}:{min}")
            # compute the time delayed
            delay = time_sch_org - time_sch_new
            delay = delay.reset_index(drop = True)[0].total_seconds() / 60
            return delay
        elif year == 1970:
            # compute utility of diverted flight
            delay = - self.max_delay 
            return delay
    
    def utility(self, state):
        """Compute aggregate utility of a given state"""
        agg_util = state['util'].sum()
        return agg_util
        
    def goal_test(self, state):
        "return True if the state is terminal"
        flight_assigned = [flight for flight in state.index] 
        if len(flight_assigned) == len(self.df):
        # !!! this is not a robust way since time slot can be none
            return True
        
    def display(self):
        """ Method to show the new schedule of the flightafter solving for the problem, 
        """
        display_df = self.df.copy()
        display_df = pd.merge(self.df[['code','time_sch']], display_df, left_on= "code", right_index = True)
        display_df['time_dff'] = display_df.apply(lambda x: (x['time_sch'] - x['time_new']).total_seconds()/ 60,axis = 1 )
        display_df['time_dff'] = display_df['time_dff'].apply(lambda x: -x if x <= 0 else "diverted" )
        return display_df


In [41]:
class Node:
    """This refenece to the anima class node"""
    def __init__(self, state,parent = None, action = None, path_cost = 0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0 
        if parent:
            self.depth = parent.depth + 1
    def __repr__(self):
        return "<Node of depth {}>".format(self.depth)
    
    def __lt__(self, node):
        return self.depth < node.depth
    
    def expand(self, problem):
        """reach the nodes reachable"""
        flight_to_assign, flight_diverted = problem.actions(self.state)
        return [self.child_node(problem, action, flight_diverted) 
                  for action in flight_to_assign]

    def child_node(self, problem,action, flight_diverted):
        """ Return a Node object representing the child node        
        """
        parent_state = self.state.copy()
        next_state = problem.result(parent_state, flight_diverted, action)
        # print(next_state)
        next_node = Node(next_state, parent=self, action = action, path_cost = problem.utility(next_state))
        return next_node # node object
    
    def solution(self):
        """??? Where is the path defined???? This only gives a list of action
        that is the flight code without returning the timeslots"""
        return self.path() # pd.DataFrame
    
    def path(self):
        return self.state
    
    

In [42]:
def best_first_graph_search(problem):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""

    # caching values on each node
    # f = memoize(f, 'f')
    # adding first node
    node = Node(problem.initial)
    print(f"The airport resumed service at {problem.hour}:{problem.min:02d}")
    iterations = 1
    # applying the goal test when generating the node
    if problem.goal_test(node.state):
        iterations += 1
        return(iterations, node)

    # expand the frontier based on the priority queue
    # the current best candidate for extension
    frontier = list()
    frontier.append(tuple([node.path_cost,node ]))
    # append the generated node to the frontier
    iterations += 1
    # print(f"Exploring depth {node.depth}")
    # print(f"[BFS Chunk] Length of frontier:{len(frontier)}")

    while frontier:
        # get the next node in the frontier
        node = frontier.pop()[1]
        iterations += 1

        # applying the goal test when expanding the node
        if problem.goal_test(node.state):
            iterations += 1
            return node.state

        # add explored node to the liss: This is implicit in the set up
        # explored.add(node.state)
        # for every child in the frontier
        for child in node.expand(problem): # child is a node
            # print(f"Exploring depth {node.depth}")
            # print(f"[BFS Chunk]Considering action {child.action}")
            # print(f"Adding child of path cost {child.path_cost} to frontier")
            # print(type(child))
            frontier.append(tuple([int(child.path_cost),child]))
            # print(frontier)
            frontier.sort(reverse = True)
            
            iterations += 1
            

        iterations += 1
        
    return None
    

In [43]:
df_subset = df.tail(20).reset_index(drop = True)
resumption = df_subset.iloc[0,0] + pd.Timedelta(minutes = 30)
# initate the problem
AMS_21_tail10 = reschedule_problem(df = df_subset, n_runway=2,
                                   resume_hour= resumption.hour, resume_min=resumption.minute,
                                   max_delay = 25)
# solve the problem
solution = best_first_graph_search(AMS_21_tail10)


In [46]:
# view the result
right_df = AMS_21_tail10.df[["code", 'time_sch']]
display_df = pd.merge(solution, right_df , left_index = True, right_on = "code")
display_df = display_df.set_index(["code"])
display_df[['time_sch',"time_new","util"]]

,time_sch,time_new,util
code,,,
OR 3802 TUI fly,2023-12-21 22:30:00,1970 01 01 00:00,-25.0
LH 2310 Lufthansa,2023-12-21 22:35:00,2023 12 21 23:00,-25.0
KL 1032 KLM,2023-12-21 22:35:00,2023 12 21 23:00,-25.0
AZ 118 ITA Airways,2023-12-21 22:40:00,2023 12 21 23:05,-25.0
KL 980 KLM,2023-12-21 23:00:00,2023 12 21 23:05,-5.0
KL 1434 KLM,2023-12-21 23:00:00,2023 12 21 23:10,-10.0
KL 1706 KLM,2023-12-21 23:00:00,2023 12 21 23:10,-10.0
KL 1118 KLM,2023-12-21 23:00:00,2023 12 21 23:15,-15.0
KL 1136 KLM,2023-12-21 23:05:00,2023 12 21 23:15,-10.0
